In [14]:
import os
from typing import Optional, Any
from graphviz import Digraph
from IPython.display import display
%matplotlib inline


# Domain definition

In [2]:
# Automechanic domain
target_transduser_file = '../domains/automechanic/recipe.txt'
manuf_resources = ['../domains/automechanic/filter_service.txt',
                   '../domains/automechanic/engine_service.txt',
                   '../domains/automechanic/wheels_service.txt']

inputs_file = '../domains/automechanic/input_clockw.txt'

# Classes definition
## Transducers

In [3]:
class State:
    """
    Transducer's automaton state representation
    """
    def __init__(self, name)-> None:
        self.name = name
        
    def __repr__(self)-> str:
        return self.name
    

class Transition:
    """
    Represents state + input -> result 
    """
    def __init__(self, state: State, input, output)-> None:
        """
        Create portion of transition change
        :param state: state of automaton
        :param input: input 
        :param output: state for transition function or output value for output function
        """
        self.output = output
        self.input = input
        self.state = state
        self.name = str(state) + " + " + str(input) + " -> " + str(output)
        
    def __repr__(self)-> str:
        return self.name 
        

class Transducer:
    
    @staticmethod
    def from_file(path):
        states = dict()
        transitions = set()
        outputs = set()
        init_state = None
        
        def generate_state(name):
            s = states.get(name, None)
            if s is None:
                s = State(name)
                states[s.name] = s
            return s
        
        with open(path) as f:
            for line in f.readlines():
                line = line.strip()
                parts = line.split(" ")
                
                if parts[0] == "#" and parts[1] == "init":
                    init_state = generate_state(parts[2])
                    continue
                
                s = generate_state(parts[0])
                
                results_parts = parts[2].strip().split("->")
                s_prime = generate_state(results_parts[0])
                transitions.add(Transition(s, parts[1], s_prime))
                outputs.add(Transition(s, parts[1], results_parts[1]))
        return Transducer(os.path.splitext(os.path.basename(path))[0], init_state, transitions, outputs)
                
                
    
    def __init__(self, name: str, init_state: State, transitions_f: list, output_f: list)-> None:
        """
        
        :param inputs: set of possible inputs
        :param outputs: set of possible outputs
        :param states: set of states
        :param init_state: initial transducer state
        :param transitions_f: list of transitions for states
        :param output_f: list of transitions for outputs
        """
        super().__init__()
        self.name = name
        self.output_f = output_f
        self.transitions_f = transitions_f
        self.init_state = init_state
        self.states = {t.state for t in transitions_f}
        assert init_state is not None and init_state in self.states
        self.outputs = {t.output for t in output_f}
        self.inputs = {t.input for t in transitions_f}
    
    def get_transaction(self, state: State, inp)-> Optional[State]:
        """
        Searches through out of all defined transactions for a transaction
        :param state: current state of automaton
        :param inp: input of the transducer
        :return: state of automaton after transaction or None
        """
        for trans in self.transitions_f:
            if trans.state.name == state.name and trans.input == inp:
                return trans.output
        return None
                
    
    def get_output(self, state: State, inp) -> Optional[Any]:
        """
        Searches through out the output definitions for a statement with
        state + input -> output
        :param state: state of the automaton of the transducer
        :param inp: input of the transducer
        :return: output of output function or None
        """
        for trans in self.output_f:
            if trans.state.name == state.name and trans.input == inp:
                return trans.output
        return None


## Define game state and game error state

In [4]:
class GameState(State):
    """
    State defined as cartesian product of state of the target transducer
    and manufacturing resources transducers
    """
    def __init__(self, target_state: State, states: list)-> None:
        super().__init__(GameState._generate_state_name(target_state, states))
        self.target_state = target_state
        self.states = states
    
    @staticmethod
    def _generate_state_name(target_state: State, states: list)-> str:
        n = ",".join([state.name for state in states])
        return target_state.name + ";" + n

class GameErrorState(State):
    """
    Error state
    """
    def __init__(self)-> None:
        super().__init__('q_err')


## Produce an automaton of the target transducer

In [5]:
def get_transducer_automaton(transducer: Transducer):
    from IPython.display import Image
    dot = Digraph(comment='The Automaton of %s transducer' % transducer.name)

    for state in list(transducer.states):
        dot.node(state.name, state.name)

    for trans in list(transducer.transitions_f):
        output = transducer.get_output(trans.state, trans.input)
        dot.edge(trans.state.name, trans.output.name, trans.input + " | " + output)

    filename = '../log/%s_transducer' % transducer.name
    dot.render(filename, format='png')
    return Image(url= filename + '.png')


## Create target transducer

In [6]:
# Read target transducer
target_transducer = Transducer.from_file(target_transduser_file)
get_transducer_automaton(target_transducer)

## Create production resources P

In [18]:
P = []
for tr_definit in manuf_resources:
    transduc = Transducer.from_file(tr_definit)
    P.append(transduc)
    print(transduc.name)
    display(get_transducer_automaton(transduc))


filter_service
engine_service
wheels_service


In [8]:
# Error state
s_err = State('s_err')
out_err = "<error>"

# Safety game for orchestration problem
G = ({inputs}, {1, ..., m} Q, q0, ro)

## Build game state space and list of transitions between them
Q = S x S1 x...xSm + q_err


In [9]:
q_err = GameErrorState()
q0 = GameState(target_transducer.init_state, [prod_transd.init_state for prod_transd in P])

game_states = dict()
game_states[q_err.name] = q_err
game_transactions = dict()
explored_states = dict()

def test_target_for_state(curr_state: GameState):
    
    # If in error state stop exploring
    if curr_state == q_err:
        print("Called to explore error state. Skip")
        return
    
    # If already explored then stop
    if explored_states.get(curr_state.name) is not None:
        print("Already in explored states: %s Skip" % curr_state.name)
        return
    else:
        explored_states[curr_state.name] = curr_state
    
    for input in target_transducer.inputs:
        
        new_state = target_transducer.get_transaction(curr_state.target_state, input)
        target_output = target_transducer.get_output(curr_state.target_state, input)
        
        if new_state is None:
            new_state = s_err
            target_output = out_err
        
        # Check for res output
        for h in range(len(P)):
            res_transd = P[h]
            res_curr_state = curr_state.states[h]
            
            new_res_state = res_transd.get_transaction(res_curr_state, input)
            res_output = res_transd.get_output(res_curr_state, input)
            
            if new_res_state is None:
                new_res_state = s_err
                res_output = out_err
            
            if res_output == target_output:
                new_res_states = list(curr_state.states)
                new_res_states[h] = new_res_state
                new_game_state = GameState(new_state, new_res_states)
            else:
                new_game_state = q_err
            
            if game_states.get(new_game_state.name, None) is None:
                game_states[new_game_state.name] = new_game_state
            else:
                new_game_state = game_states[new_game_state.name]
            
            transition = Transition(curr_state, input, new_game_state)
            if game_transactions.get(transition.name, None) is None:
                game_transactions[transition.name] = transition
            
            # Explore states if automatons aren't in error state
            if new_res_state != s_err and new_state != s_err:
                test_target_for_state(new_game_state)
            
# Run game states generation from target init state
test_target_for_state(q0)

print("After game state space exploration following states were found:")
for g_s_name in game_states.keys():
    print(game_states[g_s_name])

print("\nand following transactions:")
for g_t_name in game_transactions.keys():
    print(game_transactions[g_t_name])

Already in explored states: s0;s02,s01,s03 Skip
Already in explored states: s0;s02,s01,s03 Skip
After game state space exploration following states were found:
q_err
s1;s02,s01,s03
s_err;s_err,s01,s03
s_err;s02,s01,s_err
s2;s02,s01,s03
s_err;s02,s_err,s03
s3;s02,s01,s13
s_err;s_err,s01,s13
s_err;s02,s_err,s13
s4;s02,s01,s23
s_err;s_err,s01,s23
s_err;s02,s_err,s23
s5;s02,s01,s33
s_err;s_err,s01,s33
s_err;s02,s_err,s33
s0;s02,s01,s03
s6;s02,s01,s43
s_err;s_err,s01,s43
s_err;s02,s_err,s43
s7;s02,s01,s53
s_err;s_err,s01,s53
s_err;s02,s_err,s53
s8;s02,s01,s63
s_err;s_err,s01,s63
s_err;s02,s_err,s63

and following transactions:
s0;s02,s01,s03 + check_eng -> q_err
s0;s02,s01,s03 + check_eng -> s1;s02,s01,s03
s1;s02,s01,s03 + check_eng -> s_err;s_err,s01,s03
s1;s02,s01,s03 + check_eng -> q_err
s1;s02,s01,s03 + check_eng -> s_err;s02,s01,s_err
s1;s02,s01,s03 + change_fil -> s2;s02,s01,s03
s2;s02,s01,s03 + check_eng -> s_err;s_err,s01,s03
s2;s02,s01,s03 + check_eng -> q_err
s2;s02,s01,s03 + chec

## Check if there is a controller that for P realize target transducer
Checking whether there exists a controller C
for P that realizes T can be done by solving the safety game
G defined above


In [10]:
def get_state_sets_intersection(set1:set, set2:set)-> set:
    """
    Return set of states that are in both sets
    :param set1: 
    :param set2: 
    :return: set of states
    """
    return set1.intersection(set2)

def has_transitions_to_states(state: State, states: set) -> bool:
    """
    Check PreC(states) condition:
    Return True that for any input has at least one transition from given state to
    a state in states
    :param state: given state to check
    :param states: Win states to check transition to
    :return: True if has transition, False otherwise
    """
    
    for g_t_name in game_transactions.keys():
        trans = game_transactions[g_t_name]
        if trans.state.name == state.name:
            to_state = trans.output
            for other in states:
                if to_state.name == other.name:
                    return True
    return False

prev_states_count = -1
win_states = set(game_states.values())
# We can rely on winning set size during searching for greatest fix point since:
# 1. We start from all states of the safety game when state is by definition a cartesian product
# of the state of the target transducer, states of manufacturing resources and special
# error state q_err
# 2. All states are immutable and stored in a set thus they are unique
# 3. On every iteration we only remove states that don't match PreC precondition
while prev_states_count != len(win_states):
    prev_states_count = len(win_states)
    new_win_states = set()
    for state in win_states:
        if has_transitions_to_states(state, win_states):
            new_win_states.add(state)
        else:
            print("Drop state %s since it doesn't have any appropriate transitions" % state.name)
    win_states = get_state_sets_intersection(win_states, new_win_states)
    print("New Win set is \n%s" % "\n".join(state.name for state in new_win_states))

if len(win_states) == 0:
    print("Win set is empty: there is no controller for P that realizes T")
else:
    print("Greatest fix point found.\nWin set is \n%s" % "\n".join(state.name for state in win_states))

Drop state s_err;s_err,s01,s03 since it doesn't have any appropriate transitions
Drop state s_err;s02,s_err,s53 since it doesn't have any appropriate transitions
Drop state s_err;s02,s01,s_err since it doesn't have any appropriate transitions
Drop state s_err;s_err,s01,s23 since it doesn't have any appropriate transitions
Drop state s_err;s_err,s01,s63 since it doesn't have any appropriate transitions
Drop state s_err;s02,s_err,s03 since it doesn't have any appropriate transitions
Drop state s_err;s02,s_err,s23 since it doesn't have any appropriate transitions
Drop state s_err;s_err,s01,s43 since it doesn't have any appropriate transitions
Drop state s_err;s_err,s01,s33 since it doesn't have any appropriate transitions
Drop state s_err;s02,s_err,s63 since it doesn't have any appropriate transitions
Drop state s_err;s_err,s01,s13 since it doesn't have any appropriate transitions
Drop state s_err;s02,s_err,s43 since it doesn't have any appropriate transitions
Drop state s_err;s02,s_err,s

# Strategy Generator
Define a strategy generator based on the winning sets Win(G). 
This is a nondeterministic transducer,where nondeterminism 
is of the “don’t-care” variety: all nondeterministic choices 
are equally good.
TG = (X × Y; Q; q0; ro; gamma)
</br>
Since by definition of the task we don't care about cost of the path we use
depth-first search for a transition to next state

## Input sequence


In [11]:
# Inputs that leed to a normal sequence of states
inputs = []
with open(inputs_file) as f:
    for line in f.readlines():
        line = line.strip()
        assert line in target_transducer.inputs
        inputs.append(line)
print("Input sequence: %s" % ", ".join(inputs))

Input sequence: check_eng, change_fil, pump_fl, pump_fr, pump_rr, pump_rl


## Collect outputs from target transducer

In [12]:
outputs = []
state = target_transducer.init_state
for inp in inputs:
    new_state = target_transducer.get_transaction(state, inp)
    outp = target_transducer.get_output(state, inp)
    
    if new_state is None:
        # Error state and output
        outputs.append(out_err)
    else:
        outputs.append(outp)
        state = new_state

print("On the given input sequence an output is %s" % ", ".join(outputs))

On the given input sequence an output is engine_ok, filter_ok, wheel_fl_ok, wheel_fr_ok, wheel_rr_ok, wheel_rl_ok


## Search for manufacturing resources pipeline that realize target output on the given input
 

In [13]:
states = [transd.init_state for transd in P]
res_indexes = []
res_output = []

def find_resource(input, output, res_states)-> Optional[int]:
    """
    Depth-first search among manufacturing resources
    :param input: 
    :param output: 
    :param res_states: 
    :return: manufacturing transducer index or None if not found
    """
    for i in range(len(P)):
        res_trans = P[i]
        res_out = res_trans.get_output(res_states[i], input)
        if res_out is None:
            res_out = out_err
        if res_out == output:
            return i
    return None

for inp_index in range(len(inputs)):
    t_input = inputs[inp_index]
    t_output = outputs[inp_index]
    
    res_index = find_resource(t_input, t_output, states)
    if res_index is None:
        print("Not able to find manufacturing resource that can produce an output %s for the input %s" % (t_output, t_input))
        print("States are %s " % ", ".join([s.name for s in states]))
        break
    
    res_indexes.append(res_index)
    res_transd = P[res_index]
    new_res_state = res_transd.get_transaction(states[res_index], t_input)
    res_out = res_transd.get_output(states[res_index], t_input)
    
    if new_res_state is None:
        res_output.append(out_err)
    else:
        res_output.append(res_out)
        states[res_index] = new_res_state
    
print("Sequence of resources: %s" % "-> ".join(str(i) + "(" + P[i].name + ")" for i in res_indexes))    

Sequence of resources: 1(engine_service)-> 0(filter_service)-> 2(wheels_service)-> 2(wheels_service)-> 2(wheels_service)-> 2(wheels_service)
